In [1]:
#Step 1: import pandas library
import pandas as pd

In [2]:
#Step 2: Load the dataset
url = "http://island.ricerca.di.unimi.it/~alfio/shared/worldcities.xlsx"
cities_df = pd.read_excel(url)

In [3]:
cities_df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140
...,...,...,...,...,...,...,...,...,...,...,...
26564,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709
26565,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491
26566,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438
26567,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159


In [4]:
import networkx as nx

In [5]:
# Create a graph
G = nx.Graph()

In [6]:
# Add nodes for each city
for _, city_data in cities_df.iterrows():
   G.add_node(city_data["city"], lat=city_data["lat"], lng=city_data["lng"], country=city_data["country"], population=city_data["population"])

In [7]:
from geopy.distance import geodesic

In [8]:
# Function to calculate distance between two cities
def calculate_distance(city1, city2):
    coords1 = (G.nodes[city1]["lat"], G.nodes[city1]["lng"])
    coords2 = (G.nodes[city2]["lat"], G.nodes[city2]["lng"])
    return geodesic(coords1, coords2).kilometers

In [ ]:
# Add edges with weights based on the specified rules
for city1 in G.nodes():
    distances = [(city2, calculate_distance(city1, city2)) for city2 in G.nodes() if city1 != city2]
    distances.sort(key=lambda x: x[1])
    
    for i in range(min(3, len(distances))):
        city2 = distances[i][0]
        weight = 2 * (i + 1)  # 2 hours for the nearest, 4 for the second, 8 for the third
        if G.nodes[city1]["country"] != G.nodes[city2]["country"]:
            weight += 2  # Additional 2 hours if the destination city is in another country
        if G.nodes[city2]["population"] > 200000:
            weight += 2  # Additional 2 hours if the destination city has more than 200,000 inhabitants
        
        G.add_edge(city1, city2, weight=weight)

In [11]:
#Print the edges with weights
for edge in G.edges(data=True):
    print(edge)

('Tokyo', 'Shibuya', {'weight': 2})
('Tokyo', 'Kamirenjaku', {'weight': 4})
('Tokyo', 'Kawaguchi', {'weight': 8})
('Jakarta', 'Bekasi', {'weight': 4})


In [12]:
import networkx as nx

In [13]:
# Assuming G is your graph with cities as nodes and edges with weights

# Function to perform depth-first search
def dfs(city, current_time, visited, min_time):
    if current_time > 80 * 24:  # 80 days in hours
        return  # Stop if the time exceeds 80 days

    visited.add(city)

    # Check if all cities are visited
    if len(visited) == len(G.nodes()):
        # Check if it's possible to return to London
        if "London" in G.neighbors(city):
            # Update the minimum time if the current time is smaller
            min_time[0] = min(min_time[0], current_time + G[city]["London"]["weight"])
        return

    # Explore neighbors
    for neighbor in G.neighbors(city):
        if neighbor not in visited:
            # Recursively call the function for the neighbor
            new_time = current_time + G[city][neighbor]["weight"]
            dfs(neighbor, new_time, visited.copy(), min_time)

# Initialize variables
min_time = [float('inf')]  # Using a list to pass by reference
start_city = "London"

# Start the DFS from London
dfs(start_city, 0, set(), min_time)

# Print the result
if min_time[0] == float('inf'):
    print("It's not possible to travel around the world in 80 days.")
else:
    print(f"The minimum time required to travel around the world starting in {start_city} is {min_time[0]} hours.")

It's not possible to travel around the world in 80 days.
